## Phi-4 RAG s Azure AI Search

Tento notebook demonštruje, ako používať Phi-4-mini a Phi-4-multimodal na generovanie s podporou vyhľadávania (RAG) pomocou Azure AI Search. Zahŕňa scenáre s jednou modalitou (iba text) aj s viacerými modalitami (text a obrázok).

**Predpoklady:**
*   Vektorový index Azure AI Search (postupujte podľa [týchto pokynov](https://learn.microsoft.com/azure/search/search-get-started-portal-import-vectors?tabs=sample-data-storage%2Cmodel-aoai%2Cconnect-data-storage) na jeho vytvorenie)
*   Nasadené koncové body Phi-4-mini ALEBO Phi-4-multimodal v Azure AI Foundry


In [ ]:
! pip install azure-ai-inference azure-search-documents

### Text-Only RAG s Phi-4-mini

Táto sekcia ukazuje, ako používať Phi-4-mini ako model na dokončenie chatu pre RAG, pričom sa používa iba text ako vstup. Zahŕňa to pripojenie k Azure AI Foundry Inference a Azure AI Search, získanie relevantných dokumentov a generovanie odpovede pomocou získaného kontextu.


In [ ]:
# Step 1: Connect to Azure AI Foundry Inference & Azure AI Search
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery

chat_client = ChatCompletionsClient(
    endpoint=os.environ["AZURE_INFERENCE_ENDPOINT"], # Phi-4-mini endpoint
    credential=AzureKeyCredential(os.environ["AZURE_INFERENCE_CREDENTIAL"]),
)

search_client = SearchClient(
    endpoint=os.environ["AZURE_SEARCH_ENDPOINT"],
    index_name=os.environ["AZURE_SEARCH_INDEX"],
    credential=AzureKeyCredential(os.environ["AZURE_SEARCH_KEY"])
)

# Step 2: Retrieve relevant documents from Azure AI Search
def retrieve_documents(query: str, top: int = 10):
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=top, fields="text_vector")
    results = search_client.search(search_text=query,vector_queries=[vector_query],select=["content"],top=top)
    return [doc["content"] for doc in results]

# Step 3: Generate answer using RAG!
def generate_rag_response(query: str):
    docs = retrieve_documents(query)
    context = "\n---\n".join(docs)
    prompt = f"""You are a helpful assistant. Use only the following context to answer the question. If the answer isn't in the context, say 'I don't know'.
    Context: {context} Question: {query} Answer:"""
    response = chat_client.complete(messages=[UserMessage(content=prompt)])
    return response.choices[0].message.content

# Example usage
user_query = "What is the capital of France?"
answer = generate_rag_response(user_query)
print(f"Q: {user_query}\nA: {answer}")


### Multi-Modal RAG s Phi-4-multimodal

Táto sekcia ukazuje, ako používať Phi-4-multimodal ako model na dokončenie chatu pre RAG, ktorý zahŕňa textový aj obrazový vstup. Zahŕňa pripojenie k Azure AI Inference a Azure AI Search, vyhľadávanie relevantných dokumentov a generovanie multimodálnej odpovede.

**Poznámka:** Môžete tiež vykonať multi-vektorové vyhľadávanie, ak máte vo svojom indexe Azure AI Search polia `text_vector` a `image_vector`.


In [ ]:
# Step 1: Connect to Azure AI Inference & Azure AI Search
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery


chat_client = ChatCompletionsClient(
    endpoint=os.environ["AZURE_INFERENCE_ENDPOINT"], #Phi-4-multimodal serverless endpoint
    credential=AzureKeyCredential(os.environ["AZURE_INFERENCE_CREDENTIAL"]),
)

search_client = SearchClient(
    endpoint=os.environ["AZURE_SEARCH_ENDPOINT"],
    index_name=os.environ["AZURE_SEARCH_INDEX"],
    credential=AzureKeyCredential(os.environ["AZURE_SEARCH_KEY"])
)

# Step 2: Retrieve relevant documents from Azure AI Search
def retrieve_documents(query: str, top: int = 3):
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=top, fields="text_vector")
    results = search_client.search(search_text=query, vector_queries=[vector_query], select=["content"], top=top)    
    return [doc["content"] for doc in results]

## Example for Muli-modal search if you have a text_vector AND image_vector field in your vector_index
## NOTE, image vectorization is in preview at the time of writing this code, please use azure-search-documents pypi version >11.6.0b6 
# def retrieve_documents_multimodal(query: str, image_url: str, top: int = 3):
#     text_vector_query = VectorizableTextQuery(
#         text=query,
#         k_nearest_neighbors=top,
#         fields="text_vector",
#         weight=0.5  # Adjust weight as needed
#     )
#     image_vector_query = VectorizableImageUrlQuery(
#         url=image_url,
#         k_nearest_neighbors=top,
#         fields="image_vector",
#         weight=0.5  # Adjust weight as needed
#     )

#     results = search_client.search(
#         search_text=query,  
#         vector_queries=[text_vector_query, image_vector_query],
#         select=["content"],
#         top=top
#     )
#     return [doc["content"] for doc in results]


# Step 3: Generate a multimodal RAG-based answer using retrieved text and an image input
def generate_multimodal_rag_response(query: str, image_url: str):
    # Retrieve text context from search
    docs = retrieve_documents(query)
    context = "\n---\n".join(docs)

    # Build a prompt that combines the retrieved context with the user query
    prompt = f"""You are a helpful assistant. Use only the following context to answer the question. If the answer isn't in the context, say 'I don't know'.
    Context: {context} Question: {query} Answer:"""
    # Create a chat request that includes both text and image input
    response = chat_client.complete(
        messages=[
            SystemMessage(content="You are a helpful assistant that can process both text and images."),
            UserMessage(
                content=[
                    TextContentItem(text=prompt),
                    ImageContentItem(image_url=ImageUrl(url=image_url, detail=ImageDetailLevel.HIGH)),
                ]
            ),
        ]
    )
    return response.choices[0].message.content

# Example usage
user_query = "Can you search for similar items to this shoe?"
sample_image_url = "https://images.unsplash.com/photo-1542291026-7eec264c27ff?q=80&w=1770&auto=format&fit=crop&ixlib=rb-4.0.3"
answer = generate_multimodal_rag_response(user_query, sample_image_url)
print(f"Q: {user_query}\nA: {answer}")


---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby na automatický preklad [Co-op Translator](https://github.com/Azure/co-op-translator). Aj keď sa snažíme o presnosť, upozorňujeme, že automatické preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho pôvodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre dôležité informácie sa odporúča profesionálny ľudský preklad. Nezodpovedáme za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
